In [8]:
from py2neo import Graph
import math


In [9]:
# Connect to your Neo4j database
neo4j_uri = "bolt://localhost:7687"  # Replace with your Neo4j server URI
#neo4j_user = "your_username"  # Replace with your Neo4j username
#neo4j_password = "your_password"  # Replace with your Neo4j password

graph = Graph(neo4j_uri)

In [10]:
# Run the Cypher query to retrieve all nodes
def get_all_nodes():
    query = "MATCH (node) RETURN node"
    result = graph.run(query)

    # Fetch and display the results
    #for record in result:
        #print(record)
    #print(result)
    return result

In [11]:
def get_shortest_path(start_name, end_name):   
    # Create a Cypher query to find the shortest path between the nodes
    cypher_query = (
        f"MATCH "
        "(TRC1:SINGLE_NODE {name: $start_name}), "
        "(TRC2:SINGLE_NODE {name: $end_name}), "
        "p = shortestPath((TRC1)-[*]-(TRC2)) "
        #"WHERE TRC1<>TRC2 "
        "RETURN length(p) AS length"
    )
    
    # Run the Cypher query and retrieve the shortest path length as the result
    result = graph.run(cypher_query, start_name=start_name, end_name=end_name).evaluate()
    
    return int(result) if result is not None else None

In [12]:
def progress_callback(longest_shortest_path, total_paths, pos, steps, progress_percentage=None, progress_percentage_total=None):
    if progress_percentage is not None:
        print(f"Total Progress: {progress_percentage_total}% - Progress with this Node: {progress_percentage}% - Longest Shortest Path: {longest_shortest_path} - Position in List: {pos}, {steps}")
    else:
        print(f"Longest Shortest Path: {longest_shortest_path}")

In [13]:
def custom_path_finding_algorithm(start, steps, progress_callback_flag=True):
    nodes_list = []
    # Get all nodes in the graph
    nodes = list(get_all_nodes())
    #print("Getting Nodes")
    
    for record in nodes:
        node = record['node']
        node_name = node.get('name') 
        nodes_list.append(node_name)
        #print(f"Adding {node_name}")


    # Initialize variables for progress monitoring
    longest_shortest_path_length = 0
    total_paths = len(nodes) - 1  # Exclude the source node
    print(f"Total Nodes: {total_paths+1}")
    num_combinations = math.factorial(len(nodes)) // (math.factorial(2) * math.factorial(len(nodes) - 2))
    print(f"Possible Paths: {num_combinations}")
    progress_percentage = 0
    progress_percentage_total = 0

    for i in range(start, len(nodes_list)): # SET STARTING POINT HERE AFTER RANGE
        source_node = nodes_list[i]
        progress_percentage = 0
        # Iterate through each node as the destination node
        for j in range(i + 1, len(nodes_list)):
            destination_node = nodes_list[j]
            steps += 1
            # Find the shortest path length from the source to the destination
            shortest_path_length = get_shortest_path(source_node, destination_node)
            #print(source_node, destination_node)

            # Check if a new longest shortest path is found
            if shortest_path_length is not None:
                if shortest_path_length > longest_shortest_path_length:
                    longest_shortest_path_length = shortest_path_length
                    if progress_callback_flag:
                        progress_callback(longest_shortest_path_length, total_paths, i, steps)

            # Check if 1% progress is made
            progress_percentage_calc = (nodes_list.index(destination_node) - 1) / total_paths * 100
            progress_percentage_new = math.ceil(progress_percentage_calc)
            if progress_percentage_new > progress_percentage and progress_callback_flag:
                progress_percentage = progress_percentage_new
                progress_percentage_total = steps / num_combinations * 100
                progress_percentage_total= math.ceil(progress_percentage_total)
                progress_callback(longest_shortest_path_length, total_paths, i, steps, progress_percentage, progress_percentage_total)
    
    # Optionally, return the result or perform any other necessary actions
    return longest_shortest_path_length




In [7]:
custom_path_finding_algorithm(2, 890688) #PARAMETERS ARE START AND STEPS

Total Nodes: 308197
Possible Paths: 47492541306
Longest Shortest Path: 1
Total Progress: 1% - Progress with this Node: 1% - Longest Shortest Path: 1 - Position in List: 2, 890689
Longest Shortest Path: 2
Longest Shortest Path: 3
Total Progress: 1% - Progress with this Node: 2% - Longest Shortest Path: 3 - Position in List: 2, 893769
Total Progress: 1% - Progress with this Node: 3% - Longest Shortest Path: 3 - Position in List: 2, 896851
Total Progress: 1% - Progress with this Node: 4% - Longest Shortest Path: 3 - Position in List: 2, 899933
Total Progress: 1% - Progress with this Node: 5% - Longest Shortest Path: 3 - Position in List: 2, 903015
Longest Shortest Path: 4
Total Progress: 1% - Progress with this Node: 6% - Longest Shortest Path: 4 - Position in List: 2, 906097
Total Progress: 1% - Progress with this Node: 7% - Longest Shortest Path: 4 - Position in List: 2, 909179
Total Progress: 1% - Progress with this Node: 8% - Longest Shortest Path: 4 - Position in List: 2, 912261
Tota

KeyboardInterrupt: 

In [14]:
nodes = list(get_all_nodes())
#print("Getting Nodes")
#print(f"{nodes}")

for record in nodes:
    node = record['node']
    node_name = node.get('name') 
    #nodes_list.append(node_name)
    print(f"{node_name}")

5 Gyres
Plastic pollution
Water pollution
Agricultural wastewater treatment
Industrial wastewater treatment
Firewater (fire fighting)
Air pollution
Gas flare
Non-exhaust emissions
Particulates
Aquatic biomonitoring
Water quality
Biological integrity
Bioassay
Bioindicator
Aquatic toxicology
Marine pollution
Effluent
Atmospheric dispersion modeling
CMAQ
HYSPLIT
Ballast water discharge and the environment
Biotic index
Biosurvey
Marine debris
Freshwater acidification
Wastewater treatment
Injection well
Volatile organic compound
Exhaust gas
Sewage treatment
Eutrophication
Sustainable Development Goal 14
Nutrient pollution
Microplastics
Pollution
Water treatment
Sanitary sewer
Septic drain field
List of wastewater treatment technologies
Marine plastic pollution
Plasticosis
Water testing
Point source pollution
Pollutant
Sewage
VOC contamination of groundwater


In [ ]:
def calculate_combinations(x):
    if x < 2:
        return 0  # Not enough objects for combinations of size 2

    # Calculate the number of combinations using the binomial coefficient formula
    num_combinations = math.factorial(x) // (math.factorial(2) * math.factorial(x - 2))
    
    return num_combinations

# Example usage
x = 10  # Replace with your actual number of objects
result = calculate_combinations(x)
print(f"The total number of combinations for a sample size of 2 from {x} objects is: {result}")